In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
batchsize=8

In [4]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'D:\Projects\Drivers Drowsiness Detection\Dataset\Prepared_Data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'D:\Projects\Drivers Drowsiness Detection\Dataset\Prepared_Data\train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 60469 images belonging to 2 classes.
Found 15117 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'D:\Projects\Drivers Drowsiness Detection\Dataset\Prepared_Data\test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 9312 images belonging to 2 classes.


In [6]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [8]:
checkpoint = ModelCheckpoint(r'D:\Projects\Drivers Drowsiness Detection\models\model_1.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [9]:
from PIL import Image
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=10)

Epoch 1/10
7558/7558 [==============================] - ETA: 0s - loss: 0.2029 - accuracy: 0.9209
Epoch 1: val_loss improved from inf to 0.22720, saving model to D:\Projects\Drivers Drowsiness Detection\models\model_1.h5
7558/7558 [==============================] - 4127s 530ms/step - loss: 0.2029 - accuracy: 0.9209 - val_loss: 0.2272 - val_accuracy: 0.9103 - lr: 0.0010
Epoch 2/10
7558/7558 [==============================] - ETA: 0s - loss: 0.1738 - accuracy: 0.9326
Epoch 2: val_loss did not improve from 0.22720
7558/7558 [==============================] - 278s 37ms/step - loss: 0.1738 - accuracy: 0.9326 - val_loss: 0.2320 - val_accuracy: 0.9009 - lr: 0.0010
Epoch 3/10
7556/7558 [============================>.] - ETA: 0s - loss: 0.1666 - accuracy: 0.9365
Epoch 3: val_loss improved from 0.22720 to 0.21664, saving model to D:\Projects\Drivers Drowsiness Detection\models\model_1.h5
7558/7558 [==============================] - 161s 21ms/step - loss: 0.1666 - accuracy: 0.9365 - val_loss: 0.2

In [ ]:
loss_tr, acc_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

In [ ]:
loss_vr, acc_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

In [ ]:
loss_test, acc_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

In [ ]:
import matplotlib.pyplot as plt



In [ ]:
plt.plot(model["loss"], 'b', label="loss")
plt.plot(model["val_loss"], 'r', label="val_loss")
plt.xlabel("epoch")
plt.ylabel("frequency")
plt.legend()
plt.show()

TypeError: 'Functional' object is not subscriptable